# Getting Started With Bokeh

### pip install bokeh

# Gapminder Details

## Imports

In [1]:
import numpy as np
import pandas as pd

from bokeh.core.properties import field
from bokeh.io import curdoc
from bokeh.layouts import layout
from bokeh.models import (
    ColumnDataSource, HoverTool, SingleIntervalTicker, Slider, Button, Label,
    CategoricalColorMapper
)
from bokeh.palettes import Spectral6
from bokeh.plotting import figure


## Get the data

In [2]:
import bokeh.sampledata
bokeh.sampledata.download()

Using data directory: /Users/vickieliu/.bokeh/data
Downloading: CGM.csv (1589982 bytes)
   1589982 [100.00%]
Downloading: US_Counties.zip (3182088 bytes)
   3182088 [100.00%]
Unpacking: US_Counties.csv
Downloading: us_cities.json (713565 bytes)
    713565 [100.00%]
Downloading: unemployment09.csv (253301 bytes)
    253301 [100.00%]
Downloading: AAPL.csv (166698 bytes)
    166698 [100.00%]
Downloading: FB.csv (9706 bytes)
      9706 [100.00%]
Downloading: GOOG.csv (113894 bytes)
    113894 [100.00%]
Downloading: IBM.csv (165625 bytes)
    165625 [100.00%]
Downloading: MSFT.csv (161614 bytes)
    161614 [100.00%]
Downloading: WPP2012_SA_DB03_POPULATION_QUINQUENNIAL.zip (5148539 bytes)
   5148539 [100.00%]
Unpacking: WPP2012_SA_DB03_POPULATION_QUINQUENNIAL.csv
Downloading: gapminder_fertility.csv (64346 bytes)
     64346 [100.00%]
Downloading: gapminder_population.csv (94509 bytes)
     94509 [100.00%]
Downloading: gapminder_life_expectancy.csv (73243 bytes)
     73243 [100.00%]
Downloadi

## data.py 

## Prepare the data

In order to create an interactive plot in Bokeh, we need to animate snapshots of the data over time from 1964 to 2013. In order to do this, we can think of each year as a separate static plot. We can then use a JavaScript callback to change the data source that is driving the plot.

為了使用 Bokeh 畫出互動式圖型，我們利用 sampledata 中 gapminder 的 1964 年至 2013 年資料，以年為單位製作靜態圖，再利用 JavaScript callback 的功能來產生想要的結果。

### JavaScript Callbacks

Bokeh exposes various callbacks, which can be specified from Python, that trigger actions inside the browser’s JavaScript runtime. This kind of JavaScript callback can be used to add interesting interactions to Bokeh documents without the need to use a Bokeh server (but can also be used in conjuction with a Bokeh server). Custom callbacks can be set using a CustomJS object and passing it as the callback argument to a Widget object.

As the data we will be using today is not too big, we can pass all the datasets to the JavaScript at once and switch between them on the client side using a slider widget.

This means that we need to put all of the datasets together build a single data source for each year. First we will load each of the datasets with the process_data() function and do a bit of clean up:

即使不使用 Bokeh 仍可以使用 JavaScript callback。由於這個資料量不大，所以可以一次性將所有資料集傳給 JavaScript 再逐年分割。使用 process_data( ) 搜集所有的資料，並對資料進行些預處理：

In [3]:
def process_data():
    from bokeh.sampledata.gapminder import fertility, life_expectancy, population, regions

    # Make the column names ints not strings for handling
    columns = list(fertility.columns)
    years = list(range(int(columns[0]), int(columns[-1]) + 1))
    rename_dict = dict(zip(columns, years))

    fertility = fertility.rename(columns=rename_dict)
    life_expectancy = life_expectancy.rename(columns=rename_dict)
    population = population.rename(columns=rename_dict)
    regions = regions.rename(columns=rename_dict)
    
    regions_list = list(regions.Group.unique())

    # Turn population into bubble sizes. Use min_size and factor to tweak.
    scale_factor = 200
    population_size = np.sqrt(population / np.pi) / scale_factor
    min_size = 3
    population_size = population_size.where(population_size >= min_size).fillna(min_size)

    return fertility, life_expectancy, population_size, regions, years, regions_list

## process_data( )

In [4]:
from bokeh.sampledata.gapminder import fertility, life_expectancy, population, regions

In [5]:
fertility

,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,...,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013
Country,,,,,,,,,,,,,,,,,,,,,
Afghanistan,7.671,7.671,7.671,7.671,7.671,7.671,7.671,7.671,7.671,7.671,...,7.136,6.930,6.702,6.456,6.196,5.928,5.659,5.39500,5.14100,4.900
Albania,5.711,5.594,5.483,5.376,5.268,5.160,5.050,4.933,4.809,4.677,...,2.004,1.919,1.849,1.796,1.761,1.744,1.741,1.74800,1.76000,1.771
Algeria,7.653,7.655,7.657,7.658,7.657,7.652,7.641,7.622,7.591,7.548,...,2.448,2.507,2.580,2.656,2.725,2.781,2.817,2.82900,2.82000,2.795
American Samoa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Andorra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Angola,7.425,7.430,7.422,7.403,7.375,7.339,7.301,7.264,7.232,7.208,...,6.704,6.657,6.598,6.523,6.434,6.331,6.218,6.09900,5.97900,5.863
Anguilla,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Antigua and Barbuda,4.250,4.193,4.125,4.042,3.942,3.824,3.684,3.517,3.328,3.122,...,2.246,2.224,2.203,2.183,2.164,2.146,2.130,2.11500,2.10200,2.089
Argentina,3.068,3.058,3.051,3.048,3.049,3.056,3.073,3.104,3.148,3.203,...,2.310,2.286,2.268,2.254,2.241,2.228,2.215,2.20100,2.18800,2.175


In [6]:
fertility.columns

Index([u'1964', u'1965', u'1966', u'1967', u'1968', u'1969', u'1970', u'1971',
       u'1972', u'1973', u'1974', u'1975', u'1976', u'1977', u'1978', u'1979',
       u'1980', u'1981', u'1982', u'1983', u'1984', u'1985', u'1986', u'1987',
       u'1988', u'1989', u'1990', u'1991', u'1992', u'1993', u'1994', u'1995',
       u'1996', u'1997', u'1998', u'1999', u'2000', u'2001', u'2002', u'2003',
       u'2004', u'2005', u'2006', u'2007', u'2008', u'2009', u'2010', u'2011',
       u'2012', u'2013'],
      dtype='object')

In [7]:
life_expectancy

,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,...,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013
Country,,,,,,,,,,,,,,,,,,,,,
Afghanistan,33.639,34.152,34.662,35.170,35.674,36.172,36.663,37.143,37.614,38.075,...,56.583,57.071,57.582,58.102,58.618,59.124,59.612,60.079,60.524,60.947
Albania,65.475,65.863,66.122,66.316,66.500,66.702,66.948,67.251,67.595,67.966,...,75.725,75.949,76.124,76.278,76.433,76.598,76.780,76.979,77.185,77.392
Algeria,47.953,48.389,48.806,49.205,49.592,49.976,50.366,50.767,51.195,51.670,...,69.682,69.854,70.020,70.180,70.332,70.477,70.615,70.747,70.874,71.000
American Samoa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Andorra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Angola,34.604,35.007,35.410,35.816,36.222,36.627,37.032,37.439,37.846,38.247,...,48.036,48.572,49.041,49.471,49.882,50.286,50.689,51.094,51.498,51.899
Anguilla,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Antigua and Barbuda,63.775,64.149,64.511,64.865,65.213,65.558,65.898,66.232,66.558,66.875,...,74.355,74.544,74.729,74.910,75.087,75.263,75.437,75.610,75.783,75.954
Argentina,65.388,65.481,65.603,65.759,65.953,66.184,66.449,66.740,67.044,67.349,...,74.645,74.843,75.036,75.225,75.410,75.592,75.772,75.950,76.128,76.305


In [8]:
population

,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,...,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013
Country,,,,,,,,,,,,,,,,,,,,,
Afghanistan,10474903.0,10697983.0,10927724.0,11163656.0,11411022.0,11676990.0,11964906.0,12273101.0,12593688.0,12915499.0,...,26693486.0,27614718.0,28420974.0,29145841.0,29839994.0,30577756.0,31411743.0,32358260.0,33397058.0,34499915.0
Albania,1817098.0,1869942.0,1922993.0,1976140.0,2029314.0,2082474.0,2135599.0,2188650.0,2241623.0,2294578.0,...,3124861.0,3141800.0,3156607.0,3169665.0,3181397.0,3192723.0,3204284.0,3215988.0,3227373.0,3238316.0
Algeria,11654905.0,11923002.0,12229853.0,12572629.0,12945462.0,13338918.0,13746185.0,14165889.0,14600659.0,15052371.0,...,32396048.0,32888449.0,33391954.0,33906605.0,34428028.0,34950168.0,35468208.0,35980193.0,36485828.0,36983924.0
American Samoa,22672.0,23480.0,24283.0,25087.0,25869.0,26608.0,27288.0,27907.0,28470.0,28983.0,...,61871.0,62962.0,64045.0,65130.0,66217.0,67312.0,68420.0,69543.0,70680.0,71834.0
Andorra,17438.0,18529.0,19640.0,20772.0,21931.0,23127.0,24364.0,25656.0,26997.0,28357.0,...,75292.0,77888.0,79874.0,81390.0,82577.0,83677.0,84864.0,86165.0,87518.0,88909.0
Angola,5337063.0,5431135.0,5524090.0,5617242.0,5713156.0,5815282.0,5926333.0,6047736.0,6179685.0,6322194.0,...,15957460.0,16489021.0,17010366.0,17525367.0,18037964.0,18555115.0,19081912.0,19618432.0,20162517.0,20714494.0
Anguilla,6057.0,6114.0,6171.0,6228.0,6287.0,6344.0,6400.0,6454.0,6506.0,6554.0,...,13083.0,13574.0,14008.0,14393.0,14737.0,15055.0,15358.0,15645.0,15911.0,16158.0
Antigua and Barbuda,58653.0,59844.0,61032.0,62226.0,63401.0,64528.0,65587.0,66554.0,67419.0,68182.0,...,82838.0,83916.0,84952.0,85935.0,86881.0,87802.0,88710.0,89612.0,90510.0,91404.0
Argentina,21966478.0,22296628.0,22622012.0,22944941.0,23273148.0,23616971.0,23983358.0,24376109.0,24791983.0,25222223.0,...,38340778.0,38681174.0,39023850.0,39368066.0,39714298.0,40062470.0,40412376.0,40764561.0,41118986.0,41473982.0


In [9]:
regions

,Group,ID
Country,,
Angola,Sub-Saharan Africa,AO
Benin,Sub-Saharan Africa,BJ
Botswana,Sub-Saharan Africa,BW
Burkina Faso,Sub-Saharan Africa,BF
Burundi,Sub-Saharan Africa,BI
Cameroon,Sub-Saharan Africa,CM
Cape Verde,Sub-Saharan Africa,CV
Central African Rep.,Sub-Saharan Africa,CF
Chad,Sub-Saharan Africa,TD


In [10]:
from bokeh.sampledata.gapminder import fertility, life_expectancy, population, regions

# Make the column names ints not strings for handling
columns = list(fertility.columns)
years = list(range(int(columns[0]), int(columns[-1]) + 1))
rename_dict = dict(zip(columns, years))

fertility = fertility.rename(columns=rename_dict)
life_expectancy = life_expectancy.rename(columns=rename_dict)
population = population.rename(columns=rename_dict)

regions_list = list(regions.Group.unique())

# Turn population into bubble sizes. Use min_size and factor to tweak.
scale_factor = 200
population_size = np.sqrt(population / np.pi) / scale_factor
min_size = 3
population_size = population_size.where(population_size >= min_size).fillna(min_size)


In [11]:
columns

[u'1964',
 u'1965',
 u'1966',
 u'1967',
 u'1968',
 u'1969',
 u'1970',
 u'1971',
 u'1972',
 u'1973',
 u'1974',
 u'1975',
 u'1976',
 u'1977',
 u'1978',
 u'1979',
 u'1980',
 u'1981',
 u'1982',
 u'1983',
 u'1984',
 u'1985',
 u'1986',
 u'1987',
 u'1988',
 u'1989',
 u'1990',
 u'1991',
 u'1992',
 u'1993',
 u'1994',
 u'1995',
 u'1996',
 u'1997',
 u'1998',
 u'1999',
 u'2000',
 u'2001',
 u'2002',
 u'2003',
 u'2004',
 u'2005',
 u'2006',
 u'2007',
 u'2008',
 u'2009',
 u'2010',
 u'2011',
 u'2012',
 u'2013']

In [12]:
years

[1964,
 1965,
 1966,
 1967,
 1968,
 1969,
 1970,
 1971,
 1972,
 1973,
 1974,
 1975,
 1976,
 1977,
 1978,
 1979,
 1980,
 1981,
 1982,
 1983,
 1984,
 1985,
 1986,
 1987,
 1988,
 1989,
 1990,
 1991,
 1992,
 1993,
 1994,
 1995,
 1996,
 1997,
 1998,
 1999,
 2000,
 2001,
 2002,
 2003,
 2004,
 2005,
 2006,
 2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013]

In [13]:
rename_dict

{u'1964': 1964,
 u'1965': 1965,
 u'1966': 1966,
 u'1967': 1967,
 u'1968': 1968,
 u'1969': 1969,
 u'1970': 1970,
 u'1971': 1971,
 u'1972': 1972,
 u'1973': 1973,
 u'1974': 1974,
 u'1975': 1975,
 u'1976': 1976,
 u'1977': 1977,
 u'1978': 1978,
 u'1979': 1979,
 u'1980': 1980,
 u'1981': 1981,
 u'1982': 1982,
 u'1983': 1983,
 u'1984': 1984,
 u'1985': 1985,
 u'1986': 1986,
 u'1987': 1987,
 u'1988': 1988,
 u'1989': 1989,
 u'1990': 1990,
 u'1991': 1991,
 u'1992': 1992,
 u'1993': 1993,
 u'1994': 1994,
 u'1995': 1995,
 u'1996': 1996,
 u'1997': 1997,
 u'1998': 1998,
 u'1999': 1999,
 u'2000': 2000,
 u'2001': 2001,
 u'2002': 2002,
 u'2003': 2003,
 u'2004': 2004,
 u'2005': 2005,
 u'2006': 2006,
 u'2007': 2007,
 u'2008': 2008,
 u'2009': 2009,
 u'2010': 2010,
 u'2011': 2011,
 u'2012': 2012,
 u'2013': 2013}

In [14]:
fertility

,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,...,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013
Country,,,,,,,,,,,,,,,,,,,,,
Afghanistan,7.671,7.671,7.671,7.671,7.671,7.671,7.671,7.671,7.671,7.671,...,7.136,6.930,6.702,6.456,6.196,5.928,5.659,5.39500,5.14100,4.900
Albania,5.711,5.594,5.483,5.376,5.268,5.160,5.050,4.933,4.809,4.677,...,2.004,1.919,1.849,1.796,1.761,1.744,1.741,1.74800,1.76000,1.771
Algeria,7.653,7.655,7.657,7.658,7.657,7.652,7.641,7.622,7.591,7.548,...,2.448,2.507,2.580,2.656,2.725,2.781,2.817,2.82900,2.82000,2.795
American Samoa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Andorra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Angola,7.425,7.430,7.422,7.403,7.375,7.339,7.301,7.264,7.232,7.208,...,6.704,6.657,6.598,6.523,6.434,6.331,6.218,6.09900,5.97900,5.863
Anguilla,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Antigua and Barbuda,4.250,4.193,4.125,4.042,3.942,3.824,3.684,3.517,3.328,3.122,...,2.246,2.224,2.203,2.183,2.164,2.146,2.130,2.11500,2.10200,2.089
Argentina,3.068,3.058,3.051,3.048,3.049,3.056,3.073,3.104,3.148,3.203,...,2.310,2.286,2.268,2.254,2.241,2.228,2.215,2.20100,2.18800,2.175


In [15]:
fertility.columns

Int64Index([1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972, 1973, 1974,
            1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985,
            1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996,
            1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007,
            2008, 2009, 2010, 2011, 2012, 2013],
           dtype='int64')

In [16]:
regions_list

[u'Sub-Saharan Africa',
 u'South Asia',
 u'Middle East & North Africa',
 u'America',
 u'Europe & Central Asia',
 u'East Asia & Pacific']

In [17]:
population_size = np.sqrt(population / np.pi) / scale_factor
population_size

,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,...,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013
Country,,,,,,,,,,,,,,,,,,,,,
Afghanistan,9.129985,9.226692,9.325238,9.425367,9.529220,9.639633,9.757751,9.882623,10.010863,10.137962,...,14.574636,14.823999,15.038847,15.229420,15.409709,15.599040,15.810335,16.046771,16.302311,16.569297
Albania,3.802632,3.857528,3.911866,3.965554,4.018553,4.070848,4.122445,4.173335,4.223538,4.273134,...,4.986668,5.000165,5.011934,5.022290,5.031576,5.040524,5.049642,5.058856,5.067802,5.076387
Algeria,9.630513,9.740649,9.865195,10.002490,10.149715,10.302802,10.458904,10.617371,10.779070,10.944540,...,16.056138,16.177700,16.301065,16.426204,16.552025,16.677068,16.800209,16.921031,17.039513,17.155428
American Samoa,0.424756,0.432259,0.439588,0.446806,0.453717,0.460152,0.465995,0.471250,0.475980,0.480249,...,0.701679,0.707839,0.713900,0.719922,0.725905,0.731882,0.737881,0.743912,0.749969,0.756067
Andorra,0.372515,0.383991,0.395335,0.406569,0.417758,0.428997,0.440321,0.451845,0.463503,0.475035,...,0.774051,0.787282,0.797256,0.804786,0.810634,0.816015,0.821782,0.828058,0.834533,0.841139
Angola,6.516978,6.574161,6.630182,6.685850,6.742689,6.802687,6.867333,6.937316,7.012587,7.092984,...,11.268781,11.454932,11.634612,11.809422,11.980883,12.151416,12.322704,12.494740,12.666815,12.839031
Anguilla,0.219545,0.220576,0.221602,0.222623,0.223675,0.224686,0.225676,0.226626,0.227537,0.228375,...,0.322663,0.328662,0.333874,0.338431,0.342452,0.346127,0.349593,0.352844,0.355831,0.358582
Antigua and Barbuda,0.683188,0.690089,0.696905,0.703689,0.710302,0.716587,0.722444,0.727750,0.732464,0.736597,...,0.811914,0.817179,0.822208,0.826952,0.831491,0.835886,0.840197,0.844458,0.848679,0.852860
Argentina,13.221334,13.320320,13.417163,13.512588,13.608888,13.709044,13.814974,13.927631,14.045937,14.167289,...,17.467290,17.544657,17.622200,17.699749,17.777411,17.855168,17.932972,18.010943,18.089071,18.166988


In [18]:
population_size = population_size.where(population_size >= min_size).fillna(min_size)
population_size

,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,...,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013
Country,,,,,,,,,,,,,,,,,,,,,
Afghanistan,9.129985,9.226692,9.325238,9.425367,9.529220,9.639633,9.757751,9.882623,10.010863,10.137962,...,14.574636,14.823999,15.038847,15.229420,15.409709,15.599040,15.810335,16.046771,16.302311,16.569297
Albania,3.802632,3.857528,3.911866,3.965554,4.018553,4.070848,4.122445,4.173335,4.223538,4.273134,...,4.986668,5.000165,5.011934,5.022290,5.031576,5.040524,5.049642,5.058856,5.067802,5.076387
Algeria,9.630513,9.740649,9.865195,10.002490,10.149715,10.302802,10.458904,10.617371,10.779070,10.944540,...,16.056138,16.177700,16.301065,16.426204,16.552025,16.677068,16.800209,16.921031,17.039513,17.155428
American Samoa,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,...,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000
Andorra,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,...,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000
Angola,6.516978,6.574161,6.630182,6.685850,6.742689,6.802687,6.867333,6.937316,7.012587,7.092984,...,11.268781,11.454932,11.634612,11.809422,11.980883,12.151416,12.322704,12.494740,12.666815,12.839031
Anguilla,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,...,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000
Antigua and Barbuda,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,...,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000
Argentina,13.221334,13.320320,13.417163,13.512588,13.608888,13.709044,13.814974,13.927631,14.045937,14.167289,...,17.467290,17.544657,17.622200,17.699749,17.777411,17.855168,17.932972,18.010943,18.089071,18.166988


In [19]:
type(fertility)

pandas.core.frame.DataFrame

## main.py

In [20]:
fertility_df, life_expectancy_df, population_df_size, \
    regions_df, years, regions_list = process_data()

In [21]:
type(fertility_df)

pandas.core.frame.DataFrame

### Panel 為 3 維數據容器：
    tems: 0 軸，每個項目對應其中一個 DataFrame。
    major_axis: 1 軸，它是每個 DataFrame 的 index。
    minor_axis: 2 軸，它是每個 DataFrame 的 column。

In [22]:
p = pd.Panel({'fertility': fertility_df, \
              'life': life_expectancy_df, 'population': population_df_size})

In [23]:
p

<class 'pandas.core.panel.Panel'>
Dimensions: 3 (items) x 244 (major_axis) x 50 (minor_axis)
Items axis: fertility to population
Major_axis axis: Afghanistan to Åland
Minor_axis axis: 1964 to 2013

In [24]:
p.to_frame()

fertility    life  population
Country     minor                               
Afghanistan 1964       7.671  33.639    9.129985
            1965       7.671  34.152    9.226692
            1966       7.671  34.662    9.325238
            1967       7.671  35.170    9.425367
            1968       7.671  35.674    9.529220
            1969       7.671  36.172    9.639633
            1970       7.671  36.663    9.757751
            1971       7.671  37.143    9.882623
            1972       7.671  37.614   10.010863
            1973       7.671  38.075   10.137962
            1974       7.671  38.529   10.258298
            1975       7.671  38.977   10.366953
            1976       7.670  39.417   10.466740
            1977       7.670  39.855   10.556305
            1978       7.670  40.298   10.622529
            1979       7.669  40.756   10.648655
            1980       7.669  41.242   10.624803
            1981       7.670  41.770   10.549022
            1982       7.671  42.347   10.430975
            1983       7.673  42.977   10.287783
            1984       7.676  43.661   10.144354
            1985       7.679  44.400   10.023417
            1986       7.681  45.192    9.922406
            1987       7.682  46.024    9.846437
            1988       7.682  46.880    9.835641
            1989       7.682  47.744    9.939463
            1990       7.687  48.601   10.183646
            1991       7.700  49.439   10.581320
            1992       7.725  50.247   11.096074
            1993       7.758  51.017   11.649256
...                      ...     ...         ...
Zimbabwe    1984       6.435  61.246    8.235569
            1985       6.223  61.400    8.394526
            1986       6.004  61.365    8.551321
            1987       5.784  61.133    8.705436
            1988       5.569  60.697    8.854472
            1989       5.365  60.049    8.995767
            1990       5.176  59.157    9.127500
            1991       5.001  57.990    9.248328
            1992       4.840  56.571    9.358445
            1993       4.690  54.952    9.459456
            1994       4.554  53.197    9.553761
            1995       4.432  51.378    9.642812
            1996       4.328  49.565    9.727686
            1997       4.240  47.839    9.807215
            1998       4.169  46.284    9.878049
            1999       4.112  44.977    9.935680
            2000       4.069  43.976    9.977337
            2001       4.039  43.310   10.003439
            2002       4.018  42.965   10.016467
            2003       4.002  42.936   10.018519
            2004       3.987  43.244   10.012528
            2005       3.969  43.958   10.001717
            2006       3.941  45.157    9.985381
            2007       3.903  46.827    9.966072
            2008       3.853  48.890    9.954207
            2009       3.792  51.234    9.963176
            2010       3.721  53.684   10.002022
            2011       3.643  56.040   10.074528
            2012       3.564  58.142   10.176422
            2013       3.486  59.871   10.298556

[10100 rows x 3 columns]

In [25]:
data = {}

region_name = regions_df.Group
region_name.name = 'region'

for year in years:
    df = pd.concat([p.loc[:, :, year], region_name], axis=1).reset_index()
    data[year] = df.to_dict('series')

source = ColumnDataSource(data=data[years[0]])

In [26]:
region_name

Country
Angola                              Sub-Saharan Africa
Benin                               Sub-Saharan Africa
Botswana                            Sub-Saharan Africa
Burkina Faso                        Sub-Saharan Africa
Burundi                             Sub-Saharan Africa
Cameroon                            Sub-Saharan Africa
Cape Verde                          Sub-Saharan Africa
Central African Rep.                Sub-Saharan Africa
Chad                                Sub-Saharan Africa
Comoros                             Sub-Saharan Africa
Congo, Dem. Rep.                    Sub-Saharan Africa
Congo, Rep.                         Sub-Saharan Africa
Cote d'Ivoire                       Sub-Saharan Africa
Equatorial Guinea                   Sub-Saharan Africa
Eritrea                             Sub-Saharan Africa
Ethiopia                            Sub-Saharan Africa
Gabon                               Sub-Saharan Africa
Gambia                              Sub-Saharan Africa
Gh

In [27]:
source

ColumnDataSource(id='d1c7cace-914d-44e9-a039-ff113160a282', ...)

In [28]:
# p.loc[:, :, year] can be p.to_frame()
for year in years:
    df = pd.concat([p.to_frame(), region_name], axis=1).reset_index()
    data[year] = df.to_dict('series')

source = ColumnDataSource(data=data[years[0]])

In [29]:
source

ColumnDataSource(id='d46ca199-93dd-4f9c-a2ad-a1f3c26a5bb9', ...)

In [30]:
# setting plot frame
plot = figure(x_range=(1, 9), y_range=(20, 100), title='Gapminder Data', plot_height=300)
plot.xaxis.ticker = SingleIntervalTicker(interval=1)
plot.xaxis.axis_label = "Children per woman (total fertility)" # 總生育率
plot.yaxis.ticker = SingleIntervalTicker(interval=20)
plot.yaxis.axis_label = "Life expectancy at birth (years)" # 預期壽命
label = Label(x=1.1, y=18, text=str(years[0]), text_font_size='70pt', text_color='#eeeeee')
plot.add_layout(label)

color_mapper = CategoricalColorMapper(palette=Spectral6, factors=regions_list)
plot.circle(
    x='fertility',
    y='life',
    size='population',
    source=source,
    fill_color={'field': 'region', 'transform': color_mapper},
    fill_alpha=0.8,
    line_color='#7c7e71',
    line_width=0.5,
    line_alpha=0.5,
    legend=field('region'),
)
plot.add_tools(HoverTool(tooltips="@index", show_arrow=False, point_policy='follow_mouse'))

In [31]:
def animate_update():
    year = slider.value + 1
    if year > years[-1]:
        year = years[0]
    slider.value = year

def slider_update(attrname, old, new):
    year = slider.value
    label.text = str(year)
    source.data = data[year]

slider = Slider(start=years[0], end=years[-1], value=years[0], step=1, title="Year")
slider.on_change('value', slider_update)

In [32]:
def animate():
    if button.label == '► Play': # unicode 特殊符號
        button.label = '❚❚ Pause'
        curdoc().add_periodic_callback(animate_update, 200)
    else:
        button.label = '► Play'
        curdoc().remove_periodic_callback(animate_update)

button = Button(label='► Play', width=60)
button.on_click(animate)

layout = layout([
    [plot],
    [slider, button],
], sizing_mode='scale_width')

curdoc().add_root(layout)
curdoc().title = "Gapminder"